This solution is based on the following: 
https://www.kaggle.com/code/zlatankr/titanic-random-forest-82-78

In [1]:
import os
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
train = pd.read_csv(os.path.join('..', 'data','titanic', 'train.csv'))
test = pd.read_csv(os.path.join('..', 'data','titanic',  'test.csv'))

In [48]:
type(train.groupby(['Name_Title', 'Pclass'])['Age'])

pandas.core.groupby.generic.SeriesGroupBy

In [47]:
train['Name_Title'] = train['Name'].apply(lambda x: x.split(',')[1]).apply(lambda x: x.split()[0])
train.groupby(['Name_Title', 'Pclass'])['Age'].mean()

Name_Title  Pclass
Capt.       1         70.000000
Col.        1         58.000000
Don.        1         40.000000
Dr.         1         43.750000
            2         38.500000
Jonkheer.   1         38.000000
Lady.       1         48.000000
Major.      1         48.500000
Master.     1          5.306667
            2          2.258889
            3          5.350833
Miss.       1         30.000000
            2         22.390625
            3         16.123188
Mlle.       1         24.000000
Mme.        1         24.000000
Mr.         1         41.580460
            2         32.768293
            3         28.724891
Mrs.        1         40.882353
            2         33.682927
            3         33.515152
Ms.         2         28.000000
Rev.        2         43.166667
Sir.        1         49.000000
the         1         33.000000
Name: Age, dtype: float64

In [19]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [40]:
train.isnull().sum(axis = 0)

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [38]:
train['Survived'].value_counts()

0    549
1    342
Name: Survived, dtype: int64

In [7]:
cols = ['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp',
       'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked'] 

all(elem in train.columns for elem in cols)

True

In [3]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [4]:
train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [5]:
train['Survived'].groupby(train['Pclass']).mean()

Pclass
1    0.629630
2    0.472826
3    0.242363
Name: Survived, dtype: float64

## Feature Engineering

In [4]:
def names(train, test):
    for i in [train, test]:
        i['Name_Len'] = i['Name'].apply(lambda x: len(x))
        i['Name_Title'] = i['Name'].apply(lambda x: x.split(',')[1]).apply(lambda x: x.split()[0])
        del i['Name']
    return train, test

def age_impute(train, test):
    for i in [train, test]:
        i['Age_Null_Flag'] = i['Age'].apply(lambda x: 1 if pd.isnull(x) else 0)
        data = train.groupby(['Name_Title', 'Pclass'])['Age']
        i['Age'] = data.transform(lambda x: x.fillna(x.mean()))
    return train, test

def fam_size(train, test):
    for i in [train, test]:
        i['Fam_Size'] = np.where((i['SibSp']+i['Parch']) == 0 , 'Solo',
                           np.where((i['SibSp']+i['Parch']) <= 3,'Nuclear', 'Big'))
        del i['SibSp']
        del i['Parch']
    return train, test

def ticket_grouped(train, test):
    for i in [train, test]:
        i['Ticket_Lett'] = i['Ticket'].apply(lambda x: str(x)[0]).astype(str)
        # i['Ticket_Lett'] = i['Ticket_Lett'].apply(lambda x: str(x))
        i['Ticket_Lett'] = np.where((i['Ticket_Lett']).isin(['1', '2', '3', 'S', 'P', 'C', 'A']), i['Ticket_Lett'],
                                   np.where((i['Ticket_Lett']).isin(['W', '4', '7', '6', 'L', '5', '8']),
                                            'Low_ticket', 'Other_ticket'))
        i['Ticket_Len'] = i['Ticket'].apply(lambda x: len(x))
        del i['Ticket']
    return train, test

def cabin(train, test):
    for i in [train, test]:
        i['Cabin_Letter'] = i['Cabin'].apply(lambda x: str(x)[0])
        del i['Cabin']
    return train, test

def cabin_num(train, test):
    for i in [train, test]:
        i['Cabin_num1'] = i['Cabin'].apply(lambda x: str(x).split(' ')[-1][1:])
        i['Cabin_num1'].replace('an', np.NaN, inplace = True)
        i['Cabin_num1'] = i['Cabin_num1'].apply(lambda x: int(x) if not pd.isnull(x) and x != '' else np.NaN)
        i['Cabin_num'] = pd.qcut(train['Cabin_num1'],3)
    train = pd.concat((train, pd.get_dummies(train['Cabin_num'], prefix = 'Cabin_num')), axis = 1)
    test = pd.concat((test, pd.get_dummies(test['Cabin_num'], prefix = 'Cabin_num')), axis = 1)
    del train['Cabin_num']
    del test['Cabin_num']
    del train['Cabin_num1']
    del test['Cabin_num1']
    return train, test

def embarked_impute(train, test):
    for i in [train, test]:
        i['Embarked'] = i['Embarked'].fillna('S')
    return train, test

def dummies(train, test, columns = ['Pclass', 'Sex', 'Embarked', 'Ticket_Lett', 'Cabin_Letter', 'Name_Title', 'Fam_Size']):
    for column in columns:
        train[column] = train[column].apply(lambda x: str(x))
        test[column] = test[column].apply(lambda x: str(x))
        # good_cols = [column+'_'+i for i in train[column].unique() if i in test[column].unique()]
        # enc = OneHotEncoder(handle_unknown='ignore', sparse=False)
        # enc.fit(train[['Name_Title', 'Age']])
        # train = pd.concat((train, pd.get_dummies(train[column], prefix = column)[good_cols]), axis = 1)
        # test = pd.concat((test, pd.get_dummies(test[column], prefix = column)[good_cols]), axis = 1)
        # del train[column]
        # del test[column]
    enc = OneHotEncoder(handle_unknown='ignore', sparse=False)
    enc.fit(train[columns])
    new_cols_train = enc.transform(train[columns])
    new_cols_test = enc.transform(test[columns])
    train = pd.concat((train, pd.DataFrame.from_records(new_cols_train, columns=enc.get_feature_names(columns))), axis=1)
    test = pd.concat((test, pd.DataFrame.from_records(new_cols_test, columns=enc.get_feature_names(columns))), axis=1)
    for column in columns:
        del train[column]
        del test[column]

    return train, test
    

def drop(train, test, bye = ['PassengerId']):
    for i in [train, test]:
        for z in bye:
            del i[z]
    return train, test



In [5]:
train = pd.read_csv(os.path.join('..', 'data','titanic', 'train.csv'))
test = pd.read_csv(os.path.join('..', 'data','titanic',  'test.csv'))
test['Fare'].fillna(train['Fare'].mean(), inplace = True)
train, test = names(train, test)
train, test = age_impute(train, test)
train, test = cabin_num(train, test)
train, test = cabin(train, test)
train, test = embarked_impute(train, test)
train, test = fam_size(train, test)
test['Fare'].fillna(train['Fare'].mean(), inplace = True)
train, test = ticket_grouped(train, test)
train, test = dummies(train, test, columns = ['Pclass', 'Sex', 'Embarked', 'Ticket_Lett',
                                                                     'Cabin_Letter', 'Name_Title', 'Fam_Size'])
train, test = drop(train, test)

c:\Users\diego\anaconda3\envs\tmf_env\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
c:\Users\diego\anaconda3\envs\tmf_env\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [14]:
train

,Survived,Age,Fare,Name_Len,Age_Null_Flag,"Cabin_num_(1.999, 28.667]","Cabin_num_(28.667, 65.667]","Cabin_num_(65.667, 148.0]",Ticket_Len,Pclass_3,...,Name_Title_Mrs.,Name_Title_Miss.,Name_Title_Master.,Name_Title_Rev.,Name_Title_Dr.,Name_Title_Ms.,Name_Title_Col.,Fam_Size_Nuclear,Fam_Size_Solo,Fam_Size_Big
0,0,22.000000,7.2500,23,0,0,0,0,9,1,...,0,0,0,0,0,0,0,1,0,0
1,1,38.000000,71.2833,51,0,0,0,1,8,0,...,1,0,0,0,0,0,0,1,0,0
2,1,26.000000,7.9250,22,0,0,0,0,16,1,...,0,1,0,0,0,0,0,0,1,0
3,1,35.000000,53.1000,44,0,0,0,1,6,0,...,1,0,0,0,0,0,0,1,0,0
4,0,35.000000,8.0500,24,0,0,0,0,6,1,...,0,0,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
886,0,27.000000,13.0000,21,0,0,0,0,6,0,...,0,0,0,1,0,0,0,0,1,0
887,1,19.000000,30.0000,28,0,0,1,0,6,0,...,0,1,0,0,0,0,0,0,1,0
888,0,16.123188,23.4500,40,1,0,0,0,10,1,...,0,1,0,0,0,0,0,1,0,0
889,1,26.000000,30.0000,21,0,0,0,1,6,0,...,0,0,0,0,0,0,0,0,1,0


In [15]:
test

,Age,Fare,Name_Len,Age_Null_Flag,"Cabin_num_(1.999, 28.667]","Cabin_num_(28.667, 65.667]","Cabin_num_(65.667, 148.0]",Ticket_Len,Pclass_3,Pclass_1,...,Name_Title_Mrs.,Name_Title_Miss.,Name_Title_Master.,Name_Title_Rev.,Name_Title_Dr.,Name_Title_Ms.,Name_Title_Col.,Fam_Size_Nuclear,Fam_Size_Solo,Fam_Size_Big
0,22.000000,7.8292,16,0,0,0,0,6,1,0,...,0,0,0,0,0,0,0,0,1,0
1,38.000000,7.0000,32,0,0,0,1,6,1,0,...,1,0,0,0,0,0,0,1,0,0
2,26.000000,9.6875,25,0,0,0,0,6,0,0,...,0,0,0,0,0,0,0,0,1,0
3,35.000000,8.6625,16,0,0,0,1,6,1,0,...,0,0,0,0,0,0,0,0,1,0
4,35.000000,12.2875,44,0,0,0,0,7,1,0,...,1,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
413,32.768293,8.0500,18,1,0,0,0,9,1,0,...,0,0,0,0,0,0,0,0,1,0
414,44.000000,108.9000,28,0,0,0,0,8,0,1,...,0,0,0,0,0,0,0,0,1,0
415,33.515152,7.2500,28,0,0,0,0,18,1,0,...,0,0,0,0,0,0,0,0,1,0
416,34.000000,8.0500,19,1,0,0,0,6,1,0,...,0,0,0,0,0,0,0,0,1,0


In [6]:
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(criterion='gini', 
                             n_estimators=700,
                             min_samples_split=10,
                             min_samples_leaf=1,
                             max_features='auto',
                             oob_score=True,
                             random_state=1,
                             n_jobs=-1)
rf.fit(train.iloc[:, 1:], train.iloc[:, 0])
print("%.4f" % rf.oob_score_)

c:\Users\diego\anaconda3\envs\tmf_env\lib\site-packages\sklearn\ensemble\_forest.py:427: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(


0.8361


In [7]:
from sklearn.metrics import classification_report
print(classification_report(rf.predict(train.iloc[:, 1:]), train.iloc[:, 0]))

              precision    recall  f1-score   support

           0       0.96      0.91      0.93       582
           1       0.84      0.93      0.88       309

    accuracy                           0.92       891
   macro avg       0.90      0.92      0.91       891
weighted avg       0.92      0.92      0.92       891



## Feature importance

In [9]:
pd.concat((pd.DataFrame(train.iloc[:, 1:].columns, columns = ['variable']), 
           pd.DataFrame(rf.feature_importances_, columns = ['importance'])), 
          axis = 1).sort_values(by='importance', ascending = False)[:20]

,variable,importance
12,Sex_female,0.111215
11,Sex_male,0.109769
33,Name_Title_Mr.,0.109746
1,Fare,0.088209
2,Name_Len,0.087904
0,Age,0.078651
8,Pclass_3,0.043268
35,Name_Title_Miss.,0.031292
7,Ticket_Len,0.031079
34,Name_Title_Mrs.,0.028852


In [7]:
predictions = rf.predict(test)
predictions = pd.DataFrame(predictions, columns=['Survived'])
# test = pd.read_csv(os.path.join('data','titanic',  'test.csv'))
predictions = pd.concat((test.iloc[:, 0], predictions), axis = 1)
predictions

,Age,Survived
0,22.000000,0
1,38.000000,1
2,26.000000,0
3,35.000000,0
4,35.000000,1
...,...,...
413,32.768293,0
414,44.000000,1
415,33.515152,0
416,34.000000,0


In [13]:
!pip install pandera

     -------------------------------------- 152.4/152.4 kB 9.5 MB/s eta 0:00:00
